In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
import scanpy.api as sc
from igraph import *
from MulticoreTSNE import MulticoreTSNE as TSNE #faster TSNE alternative
from anndata import read_h5ad
sc.logging.print_versions()
#results_file = './write/p16.h5ad'


# Load data

In [ ]:
# adata_raw = read_h5ad('./write/maca.h5ad')


In [ ]:
adata = read_h5ad('./write/maca-droplet.processed.h5ad')

In [ ]:
# Preprocessing

In [ ]:
# sc.pp.filter_genes(adata, min_cells=5)
# sc.pp.filter_cells(adata, min_genes=250)

In [ ]:
adata

In [ ]:
# # add the total counts per cell as observations-annotation to adata
# adata.obs['n_counts'] = np.sum(adata.X, axis=1).A1

In [ ]:
# adata

In [ ]:
# axs = sc.pl.violin(adata, ['n_genes', 'n_counts'],
#                    jitter=0.4, multi_panel=True)

In [ ]:
# ax = sc.pl.scatter(adata, x='n_counts', y='n_genes')

In [ ]:
# adata.raw = sc.pp.log1p(adata, copy=True) # freezes the state of the AnnData object returned by sc.pp.log1p
# sc.pp.normalize_per_cell(adata, counts_per_cell_after=1e4) #simple lib size normalization?

In [ ]:
# filter_result = sc.pp.filter_genes_dispersion(
#     adata.X, min_mean=0.0125, max_mean=10, min_disp=0.5)
# sc.pl.filter_genes_dispersion(filter_result)

In [ ]:
# adata = adata[:, filter_result.gene_subset]

In [ ]:
# sc.pp.log1p(adata)

In [ ]:
# sc.pp.scale(adata, max_value=10)

In [ ]:
#adata.write(results_file)

# Exploration


## Choose a tissue

In [ ]:
tissues_available = list(set(adata.obs['tissue']))
tissues_available.sort()
pd.DataFrame(tissues_available,columns=['Tissues'])

In [ ]:
tissofinterest = tissues_available[6]
tiss = adata[adata.obs['tissue'] == tissofinterest,:]
set(tiss.obs['age'])


In [ ]:
# tiss2 = tiss.obs[tiss.obs['Age'] == '3m']
# tiss3 = tiss.obs[tiss.obs['Age'] == '24m']
# tiss4 = tiss.obs[tiss.obs['Age'] == '18m']
# tiss5 = tiss.obs[tiss.obs['Age'] == '21m']

In [ ]:
tiss

In [ ]:
# tissaux = tiss2.append(tiss3)
# tissaux = tissaux.append(tiss4)
# tissaux = tissaux.append(tiss5)
# tiss = adata[adata.obs.index.isin(tissaux.index),:]

In [ ]:
# tiss

In [ ]:
tiss.raw = tiss

## PCA

In [ ]:
sc.tl.pca(tiss)

In [ ]:
tiss

In [ ]:
ax = sc.pl.pca_scatter(tiss, color=['tissue'], right_margin=0.5)

In [ ]:
ax = sc.pl.pca_scatter(tiss, color=['age'], right_margin=0.5)

In [ ]:
ax = sc.pl.pca_scatter(tiss, color=['sex'], right_margin=0.5)

In [ ]:
ax = sc.pl.pca_scatter(tiss, color='n_counts', right_margin=0.5)

In [ ]:
sc.pl.pca_variance_ratio(tiss, log=True)

## Louvain clustering

In [ ]:
sc.pp.neighbors(tiss, n_neighbors=15)#, method='gauss')
sc.tl.louvain(tiss, resolution = 0.2)

In [ ]:
tiss

## UMAP

In [ ]:
sc.tl.umap(tiss)

In [ ]:
sc.settings.set_figure_params(dpi=200)
sc.pl.umap(tiss, color=['tissue'], right_margin=0.5, save = '_'+ str(tissofinterest) +'_all-ages_droplet_tissue.pdf')

In [ ]:
sc.settings.set_figure_params(dpi=200)
sc.pl.umap(tiss, color=['louvain'], right_margin=0.5, save = '_'+ str(tissofinterest) +'_all-ages_droplet_louvain.pdf')

In [ ]:
sc.settings.set_figure_params(dpi=200)
sc.pl.umap(tiss, color=['age'], right_margin=0.5, save = '_'+ str(tissofinterest) +'_all-ages_droplet_age.pdf')

## tSNE

In [ ]:
sc.tl.tsne(tiss, perplexity=50)

In [ ]:
sc.settings.set_figure_params(dpi=200)
sc.pl.tsne(tiss, color=['tissue'], right_margin=0.5, save = '_'+ str(tissofinterest) +'_all-ages_droplet_tissue.pdf')

In [ ]:
sc.pl.tsne(tiss, color=['sex'], right_margin=0.5, save = '_'+ str(tissofinterest) +'_all-ages_droplet_sex.pdf')

In [ ]:
sc.pl.tsne(tiss, color=['age'], right_margin=0.5, save = '_'+ str(tissofinterest) +'_all-ages_droplet_age.pdf')

In [ ]:
sc.pl.tsne(tiss, color=['louvain'],right_margin=0.5, save = '_'+ str(tissofinterest) +'_all-ages_droplet_louvain.pdf')

## read the annotations from tabula muris

In [ ]:
maca10x3metadata2 = pd.read_csv('/data/maca/data/10x/TM_droplet_metadata.csv', low_memory=False)
#maca10x3metadata2 = maca10x3metadata2.rename(columns = {'cell':'Cell'})
len(maca10x3metadata2)

In [ ]:
ageofinterest = "3m"
tissage = tiss[tiss.obs['age'] == ageofinterest,:]
tissage

In [ ]:
# methodofinterest = "droplet"
# tissage = tiss[tiss.obs['method'] == methodofinterest,:]
# tissage

In [ ]:
tissage.obs.head()

In [ ]:
maca10x3metadata2.head()

In [ ]:
merged_left = pd.merge(left=tissage.obs,right=maca10x3metadata2, left_on='cell', right_on='cell',how = 'left')

merged_left.head()

#create a list of column headings
cols = list(merged_left.columns)


In [ ]:
# merged_inner = pd.merge(left=tissage.obs,right=maca10x3metadata2, left_on='cell', right_on='cell')
# #merged_inner = merged_inner[['Age','Cell','Channel','Sex','Tissue','batch','n_genes','n_counts','cell_ontology_class','free_annotation']]
# merged_inner.head()

In [ ]:
tissage.obs["cell_ontology_class"] = merged_left["cell_ontology_class_y"]
tissage.obs["cell_ontology_id"] = merged_left["cell_ontology_id"]
tissage.obs["free_annotation"] = merged_left["free_annotation"]
display(set(tissage.obs['cell_ontology_class']))

tissage.obs = tissage.obs.reset_index()
tissage.obs["cell_ontology_class"] = merged_left["cell_ontology_class_y"]
tissage.obs["cell_ontology_id"] = merged_left["cell_ontology_id"]
tissage.obs["free_annotation"] = merged_left["free_annotation"]
set(tissage.obs['cell_ontology_class'])


In [ ]:
# tissage.obs = tissage.obs.reset_index()

In [ ]:
# tissage.obs["cell_ontology_class"] = merged_inner["cell_ontology_class"]
# tissage.obs["free_annotation"] = merged_inner["free_annotation"]
# tissage.obs

In [ ]:
# tissage.obs.head()

In [ ]:
#sc.tl.pca(tissage)
#sc.pp.neighbors(tissage, n_neighbors=15)#, method='gauss')
#sc.tl.louvain(tissage, resolution = 0.3)
#sc.tl.tsne(tissage, perplexity=50)

In [ ]:
sc.pl.umap(tissage, color=['louvain'],right_margin=0.5)

In [ ]:
sc.pl.umap(tissage, color=['cell_ontology_class'],right_margin=0.5)

In [ ]:
sc.pl.umap(tissage, color=['free_annotation'],right_margin=0.5)

In [ ]:
df = tissage.obs[tissage.obs['louvain'].str.match('2')]['cell_ontology_class']
df.reset_index()
df = df.reset_index()
display(df.groupby('cell_ontology_class').count())
display(df.groupby('cell_ontology_class').count().sum())
display(df.groupby('cell_ontology_class').count()/df.groupby('cell_ontology_class').count().sum())
dfaux = df.groupby('cell_ontology_class').count()/df.groupby('cell_ontology_class').count().sum()
dfaux.reset_index()
dfaux = dfaux.reset_index()
display(dfaux[dfaux['index']>0.95][['cell_ontology_class']])
#dfdf.sum()
#df.drop_duplicates()
#tissage.obs['louvain']
#val = dfaux[dfaux['index']>0.95][['cell_ontology_class']].values[0]
#print(val)

In [ ]:
tiss_cell_ontology_class = {}
for i in range(0,tissage.obs['louvain'].nunique()):
    df = tissage.obs[tissage.obs['louvain'].str.match(str(i))]['cell_ontology_class']
    df.reset_index()
    df = df.reset_index()
    #df.groupby('cell_ontology_class').count()
    #df.groupby('cell_ontology_class').count().sum()
    #df.groupby('cell_ontology_class').count()/df.groupby('cell_ontology_class').count().sum()
    dfaux = df.groupby('cell_ontology_class').count()/df.groupby('cell_ontology_class').count().sum()
    dfaux.reset_index()
    dfaux = dfaux.reset_index()
    #display(dfaux[dfaux['index']>0.95][['cell_ontology_class']])
    #dfdfaux = pd.concat([dfdfaux,dfaux[dfaux['index']>0.95][['cell_ontology_class']]])
    a = dfaux[dfaux['index']>0.95][['cell_ontology_class']]
    if a.empty:
        tiss_cell_ontology_class[i] = 'nan'
    else:
        tiss_cell_ontology_class[i] = a.values[0]

tiss_cell_ontology_class



In [ ]:
tiss_free_annotation = {}
for i in range(0,tissage.obs['louvain'].nunique()):
    df = tissage.obs[tissage.obs['louvain'].str.match(str(i))]['free_annotation']
    df.reset_index()
    df = df.reset_index()
    #df.groupby('cell_ontology_class').count()
    #df.groupby('cell_ontology_class').count().sum()
    #df.groupby('cell_ontology_class').count()/df.groupby('cell_ontology_class').count().sum()
    dfaux = df.groupby('free_annotation').count()/df.groupby('free_annotation').count().sum()
    dfaux.reset_index()
    dfaux = dfaux.reset_index()
    #display(dfaux[dfaux['index']>0.95][['cell_ontology_class']])
    #dfdfaux = pd.concat([dfdfaux,dfaux[dfaux['index']>0.95][['cell_ontology_class']]])
    a = dfaux[dfaux['index']>0.95][['free_annotation']]
    if a.empty:
        tiss_free_annotation[i] = 'nan'
    else:
        tiss_free_annotation[i] = a.values[0]

tiss_free_annotation


In [ ]:
tiss.obs['louvain'] = tiss.obs['louvain'].apply(pd.to_numeric)
tiss.obs['cell_ontology_class'] = tiss.obs['louvain'].map(tiss_cell_ontology_class)
tiss.obs.head()

In [ ]:
tiss.obs['free_annotation'] = tiss.obs['louvain'].map(tiss_free_annotation)
tiss.obs.head()

In [ ]:
sc.pl.tsne(tiss, color=['louvain'],right_margin=0.5)

In [ ]:
sc.pl.tsne(tiss, color=['cell_ontology_class'],right_margin=0.5)

In [ ]:
sc.pl.tsne(tiss, color=['age'],right_margin=0.5)

In [ ]:
sc.pl.umap(tiss, color=['cell_ontology_class'],right_margin=0.5)

In [ ]:
sc.pl.umap(tiss, color=['free_annotation'],right_margin=0.5)

In [ ]:
sc.pl.umap(tiss, color=['age'],right_margin=0.5)

In [ ]:
def _build_subplots(n):
    '''
    Build subplots grid
    n: number of subplots
    '''
    nrow = int(np.sqrt(n))
    ncol = int(np.ceil(n / nrow))
    fig, axs = plt.subplots(nrow, ncol, dpi=100, figsize=(ncol*5, nrow*5))
    fig.subplots_adjust(hspace=.9)

    return fig, axs, nrow, ncol

In [ ]:
ages = ['3m','18m','21m','24m']#

fig, axs, nrow, ncol = _build_subplots(len(ages))

axs = axs.ravel()

for i in range(nrow*ncol):
    age = ages[i]
    axs[i].set_title(ages[i])
    sc.pl.scatter(tiss[:,'Cdkn2a'],ax=axs[i],show=False)


In [ ]:
ax = sc.pl.umap(tiss[tiss.var_names=='Cdkn2a'], color = ['age'])
ax = sc.pl.umap(tiss[tiss.obs['age']=='18m'], color = ['Cdkn2a'])
ax = sc.pl.umap(tiss[tiss.obs['age']=='21m'], color = ['Cdkn2a'])
ax = sc.pl.umap(tiss[tiss.obs['age']=='24m'], color = ['Cdkn2a'])

In [ ]:


sc.pl.dotplot(tiss, ['Cdkn2a','Cd4','Mmp12','Fabp4','Timp1','Il6'], groupby='age', log=False)
# dotplot options
# use_raw=None, log=False, num_categories=7, color_map='Reds', 
# figsize=None, var_group_positions=None, var_group_labels=None, 
# var_group_rotation=None, show=None, save=None, **kwds)

In [ ]:
#ax = sc.pl.stacked_violin(tiss, 'Cdkn2a', groupby='age')
sc.pl.violin(tiss, 'Cdkn2a', groupby='age', order = ['3m','18m','21m','24m'], log= False, size = 2)

In [ ]:
sc.pl.heatmap(tiss, 'Cdkn2a')

In [ ]:
tiss.var_names


In [ ]:
inflammation_markers = ['A2m','Abr','Ace','Ackr1','Ackr2','Acod1','Acp5','Ada','Adam8','Adam17',
                        'Adamts12','Adcyap1','Adipoq','Adora1','Adora2a','Adora2b','Adora3',
                        'Adra2a','Adrb2','Afap1l2','Ager','Agt','Agtr1a','Agtr1b','Agtr2','Ahcy',
                        'Ahsg','Aim2','Aimp1','Ak7','Akt1','Alox5','Alox5ap','Ankrd42','Ano6',
                        'Anxa1','Aoah','Aoc3','Apoa1','Apod','Apoe','Ash1l','Atrn','Axl']
                        
                        
                        
                        
                        
                        
                        
                        
                        
                        
                        

## finding marker genes

In [ ]:
sc.tl.rank_genes_groups(tiss, 'louvain')
sc.pl.rank_genes_groups(tiss, n_genes=20)#, save='louvain_clusters_top_genes.pdf')
#adata.write(results_file)

In [ ]:
sc.pl.rank_genes_groups_violin(tiss, n_genes=8, groups=['1'])

In [ ]:
#sc.tl.rank_genes_groups(tiss, 'louvain', method='logreg')
sc.pl.rank_genes_groups(tiss, n_genes=20)

### Show the 10 top ranked genes per cluster in a dataframe.

In [ ]:
pd.DataFrame(tiss.uns['rank_genes_groups']['names']).head(10)

### Get a table with scores and groups.



In [ ]:
result = tiss.uns['rank_genes_groups']
groups = result['names'].dtype.names
pd.DataFrame({group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names', 'scores']}).head(5)

In [ ]:
sc.pl.violin(tiss, ['Cdkn2a','Myoc','Gpx3'],groupby='Age', rotation=90)#, save='.pdf')

In [ ]:

sc.pl.violin(tiss[tiss.obs['Age']=='3m',:], ['Cdkn2a'],groupby='cell_ontology_class',rotation=90)#, save='.pdf')


In [ ]:
sc.pl.violin(tiss[tiss.obs['Age']=='18m',:], ['Cdkn2a'],groupby='cell_ontology_class', rotation=90)#, save='.pdf')


In [ ]:
sc.pl.violin(tiss[tiss.obs['Age']=='21m',:], ['Cdkn2a'],groupby='cell_ontology_class', rotation=90)#, save='.pdf')


In [ ]:
sc.pl.violin(tiss[tiss.obs['Age']=='24m',:], ['Cdkn2a'],groupby='cell_ontology_class', rotation=90)#, save='.pdf')


In [ ]:
def _build_subplots(n):
    '''
    Build subplots grid
    n: number of subplots
    '''
    nrow = int(np.sqrt(n))
    ncol = int(np.ceil(n / nrow))
    fig, axs = plt.subplots(nrow, ncol, dpi=100, figsize=(ncol*5, nrow*5))
    fig.subplots_adjust(hspace=.9)

    return fig, axs, nrow, ncol

In [ ]:
list(set(tiss.obs['Age']))

In [ ]:
ages = ['3m','18m','24m']#,'21m'

fig, axs, nrow, ncol = _build_subplots(len(ages))

axs = axs.ravel()

for i in range(nrow*ncol):
    age = ages[i]
    axs[i].set_title(ages[i])
    sc.pl.violin(tiss[tiss.obs['Age']==ages[i],:], ['Cdkn2a'],
                 groupby='cell_ontology_class', 
                 rotation=90,
                 ax=axs[i],show=False)

fig.savefig('figures/%s_p16.pdf' % tissofinterest, bbox_inches='tight')

In [ ]:
f, axarr = plt.subplots(2,2)
axarr[0,0] = sc.pl.violin(tiss[tiss.obs['Age']=='24m',:], ['Cdkn2a'],groupby='cell_ontology_class', rotation=90)
#axarr[0,1].sc.pl.violin(tiss[tiss.obs['Age']=='24m',:], ['Cdkn2a'],groupby='cell_ontology_class', rotation=90)
#axarr[1,0].sc.pl.violin(tiss[tiss.obs['Age']=='24m',:], ['Cdkn2a'],groupby='cell_ontology_class', rotation=90)
#axarr[1,1].sc.pl.violin(tiss[tiss.obs['Age']=='24m',:], ['Cdkn2a'],groupby='cell_ontology_class', rotation=90)

In [ ]:
sc.pl.scatter(n_genes, n_cells,color='Age', rotation=90)#, save='.pdf')


In [ ]:
sc.tl.rank_genes_groups(tiss, 'cell_ontology_class')
sc.pl.rank_genes_groups(tiss, n_genes=20, save='cell_types_top_genes.pdf')

In [ ]:
sc.tl.rank_genes_groups(tiss, 'age')
sc.pl.rank_genes_groups(tiss, n_genes=20, save='age_top_genes.pdf')

### subset for cell ontology

In [ ]:
tiss.obs[tiss.obs['cell_ontology_class'] == tiss.obs['cell_ontology_class'][1]].head()

In [ ]:
subtiss = tiss[tiss.obs['cell_ontology_class'] == tiss.obs['cell_ontology_class'][3],:]
sc.tl.rank_genes_groups(subtiss, 'Age', groups=['24m'], reference='3m')#, method='logreg')
sc.pl.rank_genes_groups(subtiss, n_genes=20, groups=['24m'], save='subtiss_louvain_clusters_top_genes.pdf')

In [ ]:
sc.pl.rank_genes_groups(subtiss, n_genes=20, save='subtiss_louvain_clusters_top_genes.pdf')

## Force-directed graph

In [ ]:
sc.tl.draw_graph(adata) # be patient here...

In [ ]:
sc.pl.draw_graph(adata, color=['Tissue'])

In [ ]:
sc.pl.draw_graph(adata, color=['Age'])

In [ ]:
sc.pl.draw_graph(adata, color=['Sex'])

In [ ]:
sc.pl.draw_graph(adata, color=['louvain'])

# Pseudotime analysis

In [ ]:
sc.pp.neighbors(adata)


In [ ]:
sc.tl.diffmap(adata)

In [ ]:
sc.tl.dpt(adata, n_branchings=1)

In [ ]:
#sc.pl.diffmap(adata, color=['dpt_pseudotime', 'dpt_groups', 'age'])

In [ ]:
#sc.pl.diffmap(adata, color=['dpt_pseudotime'])

In [ ]:
#sc.pl.diffmap(adata, color=['dpt_groups'])

In [ ]:
#sc.pl.diffmap(adata, color=['Age'])

# Save processed data

In [ ]:
tiss.write('./write/maca-droplet.processed_' +  tissofinterest + '.h5ad')

In [ ]:
def export_to_csv(adata, prefix):
    pd.DataFrame(adata.X, index = adata.obs_names, columns=adata.var_names).to_csv('./write/maca-droplet.processed_' +  prefix + '-expression.csv')
    adata.obs.to_csv('./write/maca-droplet.processed_' +  prefix + '-metadata.csv')

In [ ]:
export_to_csv(tiss,tissofinterest)